In [68]:
import warnings
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier, StackingClassifier, BaggingRegressor, \
    GradientBoostingRegressor, StackingRegressor
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'catboost'

In [ ]:
parameters_ensemble = {'n_estimators': np.arange(20,101,20),
                       'max_features': np.arange(3,24,10)}

# Классификация

In [ ]:
def print_classification_model_metrics(estimator, y_test, y_pred):
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print(estimator.score(X_test, y_test))

In [ ]:
dataset = pd.read_csv("../data/zoo_preprocessed.csv")
dataset

In [ ]:
y = dataset["hair"]
y

In [ ]:
X = dataset[dataset.columns[2:dataset.shape[1]]]
X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, stratify=y)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Базовая модель DTC

In [ ]:
%%time
parameters_dtc = {'max_depth': np.arange(5,16,1)}
dtc = DecisionTreeClassifier()
dtc_base = GridSearchCV(dtc, parameters_dtc).fit(X_train, y_train)
dtc_base.best_params_

In [ ]:
print_classification_model_metrics(dtc_base, y_test, dtc_base.predict(X_test))

# Модель BaggingClassifier

In [ ]:
%%time
bag = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=dtc_base.best_params_['max_depth']))
model = GridSearchCV(bag, parameters_ensemble).fit(X_train, y_train)
model.best_params_

In [ ]:
print_classification_model_metrics(model, y_test, model.predict(X_test))

# Модель GradientBoostingClassifier

In [ ]:
%%time
gbc = GradientBoostingClassifier()
model = GridSearchCV(gbc, parameters_ensemble).fit(X_train, y_train)
model.best_params_

In [ ]:
print_classification_model_metrics(model, y_test, model.predict(X_test))

# Модель StackingClassifier

In [ ]:
%%time
model = StackingClassifier(estimators=[('bag',bag), ('gbc',gbc)],
                           final_estimator=dtc_base).fit(X_train, y_train)

In [ ]:
print_classification_model_metrics(model, y_test, model.predict(X_test))

## Изучение модели CatBoostRegressor

In [ ]:
%%time
cbr = CatBoostRegressor(learning_rate=0.15).fit(X_train, y_train)
print_regression_model_metrics(cbr, y_test, cbr.predict(X_test))

# Регрессия

In [ ]:
def print_regression_model_metrics(estimator, y_test, y_pred):
    print(f"Коэффициент детерминации: {estimator.score(X,y)}")
    print(f'MSE: {mean_squared_error(y_test, y_pred)}')
    print(f'RMSE: {mean_squared_error(y_test, y_pred, squared=False)}')
    print(f'MAE: {mean_absolute_error(y_test, y_pred)}')

In [ ]:
dataset = pd.read_csv("../data/forestfires_preprocessed.csv")
dataset

In [ ]:
y = dataset["DC"]
y

In [ ]:
X = dataset[dataset.columns[1:]]
X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=0)

## Базовая модель DTR

In [ ]:
%%time
parameters_dtr = {'max_depth': np.arange(5,16,1)}
dtr = DecisionTreeRegressor().fit(X_train, y_train)
dtr_base = GridSearchCV(dtr, parameters_dtr).fit(X_train, y_train)
print(dtr_base.best_params_)
print_regression_model_metrics(dtr_base, y_test, dtr_base.predict(X_test))

# Модель BaggingRegressor

In [ ]:
%%time
br = BaggingRegressor(base_estimator=DecisionTreeRegressor(max_depth=dtr_base.best_params_['max_depth']))
model = GridSearchCV(br, parameters_ensemble).fit(X_train, y_train)
model.best_params_

In [ ]:
print_regression_model_metrics(model, y_test, model.predict(X_test))

# Модель GradientBoostingRegressor

In [ ]:
%%time
gbr = GradientBoostingRegressor()
model = GridSearchCV(gbr, parameters_ensemble).fit(X_train, y_train)
model.best_params_

In [ ]:
print_regression_model_metrics(model, y_test, model.predict(X_test))

# Модель StackingRegressor

In [ ]:
%%time
model = StackingRegressor(estimators=[('br',br), ('gbr',gbr)],
                           final_estimator=dtr_base).fit(X_train, y_train)

In [ ]:
print_regression_model_metrics(model, y_test, model.predict(X_test))

## Изучение модели CatBoostRegressor

In [ ]:
%%time
cbr = CatBoostRegressor(learning_rate=0.15).fit(X_train, y_train)
print_regression_model_metrics(cbr, y_test, cbr.predict(X_test))
